# Medieval Chronicles: The Fallen Knight

This notebook runs a text-based RPG game with RAG-based historical context.

## Setup and Imports

In [1]:
import os
import json
from dotenv import load_dotenv

from rpg_game.orchestrator.game_orchestrator import GameOrchestrator
from rpg_game.rag.retriever import RAGRetriever, load_sample_data

# Load environment variables
load_dotenv()

True

## RAG System Setup

Set up the Retrieval-Augmented Generation system with historical data.

In [2]:
def setup_rag_system():
    """Set up the RAG system with historical data"""
    print("Setting up RAG system with historical data...")
    
    # Create data directory if it doesn't exist
    os.makedirs("./data/vector_db", exist_ok=True)
    
    # Initialize RAG retriever
    retriever = RAGRetriever()
    
    # Load sample historical data
    historical_data = load_sample_data("./data/historical_data.json")
    
    if historical_data:
        # Add documents to vector database
        retriever.add_documents(historical_data)
        print(f"Added {len(historical_data)} historical documents to RAG system")
    else:
        print("No historical data found or error loading data")
    
    return retriever

## Game Loop

The main game loop for the text-based RPG.

In [3]:
def text_based_game_loop():
    """Run a simple text-based version of the game"""
    # Set up RAG system
    setup_rag_system()
    
    # Initialize game orchestrator
    game = GameOrchestrator()
    
    # Start the game
    player_name = input("Enter your character's name: ")
    current_scene = game.start_game(player_name)
    
    print("\n" + "=" * 50)
    print(f"Welcome to Medieval Chronicles: The Fallen Knight, {player_name}!")
    print("You are accompanied by Ser Elyen, a fallen knight seeking redemption.")
    print("=" * 50 + "\n")
    
    # Game loop
    while True:
        # Display current scene
        print(f"\n== {current_scene['title']} ==\n")
        print(current_scene['description'])
        print("\n")
        
        # Display historical context if available
        if current_scene.get('historical_context'):
            print("Historical Context:")
            for item in current_scene['historical_context']:
                print(f"- {item['title']}: {item['text'][:100]}...")
            print("\n")
        
        # Display player alignment
        alignment = current_scene['player_scores']['alignment']
        relationship = current_scene['player_scores']['relationship']
        print(f"Alignment: {alignment['description']} ({alignment['law_chaos']}/{alignment['good_evil']})")
        print(f"Relationship with Ser Elyen: {relationship['description']} ({relationship['trust']}/100)")
        print("\n")
        
        # Display action choices
        print("What will you do?")
        for i, action in enumerate(current_scene['actions']):
            print(f"{chr(65+i)}) {action}")
        
        # Get player choice
        while True:
            choice = input("\nEnter your choice (A, B, C, D): ").upper()
            if choice in ['A', 'B', 'C', 'D']:
                action_index = ord(choice) - ord('A')
                if action_index < len(current_scene['actions']):
                    break
            print("Invalid choice. Please try again.")
        
        # Process player action
        action_result = game.process_player_action(action_index)
        
        # Display agent response
        print("\n" + "-" * 50)
        print(f"Ser Elyen: {action_result['agent_response']}")
        print("-" * 50 + "\n")
        
        # Check if game should continue
        if action_result['has_next_scene']:
            # Advance to next scene
            current_scene = game.advance_to_next_scene()
        else:
            # End of game or branch
            print("\nYou have reached the end of this path.")
            if input("Would you like to save your progress? (y/n): ").lower() == 'y':
                game.save_game()
                print("Game saved.")
            
            if input("Would you like to continue exploring? (y/n): ").lower() != 'y':
                break
            
            # Return to a previous scene for exploration
            current_scene = game.get_current_scene()
    
    print("\nThank you for playing Medieval Chronicles: The Fallen Knight!")

## API Key Check and Main Function

Check for the AI21 API key and provide a way to set it if missing.

In [4]:
def main():
    """Main entry point"""
    # Check if AI21 API key is set
    if not os.getenv('AI21_API_KEY'):
        print("WARNING: AI21_API_KEY environment variable is not set.")
        print("Please set your AI21 API key in a .env file or environment variable.")
        api_key = input("Enter your AI21 API key to continue: ")
        os.environ['AI21_API_KEY'] = api_key
    
    # Run the game
    text_based_game_loop()

## Run the Game

Execute this cell to start the game.

In [5]:
import os

# Set your AI21 API key
api_key = input("Enter your AI21 API key: ")
os.environ['AI21_API_KEY'] = api_key
print("✅ API key set successfully.")

Enter your AI21 API key:  v7uBaXL8sXKB4SmHsrxIOa14sYsWPcT9


✅ API key set successfully.


In [6]:
# Force reload of RAG modules to apply code changes
from reload_modules import reload_rag_modules
reload_rag_modules()
if __name__ == "__main__" or 'ipykernel' in sys.modules:
    main()
    

Add this code to your Jupyter notebook:


# Force reload of RAG modules to apply code changes
from reload_modules import reload_rag_modules
reload_rag_modules()


Run this cell before starting the game.
Reloading module: rpg_game.rag.retriever
Reloading module: rpg_game.orchestrator.game_orchestrator
Module rag_helper not loaded yet
✅ All RAG modules reloaded successfully
Now you can run the game again!
Setting up RAG system with historical data...


/Users/e.baena/dungeon/rpg_game/rag/retriever.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/Users/e.baena/dungeon/rpg_game/rag/retriever.py:33: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectordb = Chroma(


Loaded vector database with 60 documents


/Users/e.baena/dungeon/rpg_game/rag/retriever.py:65: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.vectordb.persist()


Added 10 documents to vector database
Added 10 historical documents to RAG system
Loaded vector database with 70 documents
Loaded 5 scenes from game data


Enter your character's name:  ed



Welcome to Medieval Chronicles: The Fallen Knight, ed!
You are accompanied by Ser Elyen, a fallen knight seeking redemption.


== The Broken Bell ==

You stand beneath the broken bell of the village church. The wooden beams creak above you, and dust motes dance in the rays of sunlight streaming through the stained glass windows. The bell's rope hangs limply, frayed at the end.


Alignment: Neutral Neutral (0/0)
Relationship with Ser Elyen: Moderate Trust (50/100)


What will you do?
A) Pull the rope to test the bell
B) Examine the inscription on the bell
C) Search the altar for clues
D) Leave quietly and investigate the woods



Enter your choice (A, B, C, D):  a


[2025-03-23 16:41:42 - ai21.http_client.http_client - ERROR] Calling POST https://api.ai21.com/studio/v1 failed with a non-200 response code: 401 headers: Headers({'date': 'Sun, 23 Mar 2025 20:41:42 GMT', 'content-type': 'application/json', 'content-length': '50', 'connection': 'keep-alive', 'request-id': 'c7a4c633-0e38-6216-29be-bbe765f5284a', 'via': '1.1 google', 'cf-cache-status': 'DYNAMIC', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'expect-ct': 'max-age=86400, enforce', 'referrer-policy': 'same-origin', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'server': 'cloudflare', 'cf-ray': '9250c0460d9415a3-EWR'})


Unauthorized: Failed with http status code: 401 (Unauthorized). Details: {"detail":"Forbidden: Bad or missing Apikey/JWT."}